# Grab Dimensions Data

https://app.dimensions.ai/discover/publication

This notebook handles the downloads of Dimensions data.


In [1]:
import pandas as pd
import numpy as np

## Load Journals

In [2]:
origin_folder = 'Journals_Articles'

In [3]:
journals_file = f'{origin_folder}/ABS期刊汇总_DimentionsSearch.csv'
journals = pd.read_csv(journals_file, usecols=['search title', 'search id'])
journals = journals.dropna()

In [4]:
len(journals)

1074

In [5]:
journals_index = '1-20'

In [6]:
start = int(journals_index.split('-')[0])
end = int(journals_index.split('-')[1])
journals = journals[start - 1:end]

In [7]:
journals

,search title,search id
2,Journal of Accounting and Economics,jour.1138469
3,Journal of Accounting Research,jour.1137699
4,Contemporary Accounting Research,jour.1138183
5,Review of Accounting Studies,jour.1136096
6,Abacus,jour.1327791
7,Accounting and Business Research,jour.1137863
8,Accounting Forum,jour.1143209
9,Accounting Horizons,jour.1139701
12,Behavioral Research in Accounting,jour.1143411
15,Critical Perspectives on Accounting,jour.1137906


## Request Functions

This part contains functions we need to fetch the web data and should also handle the exceptions while fetching here.

In [8]:
import requests
import json

from requests import ConnectionError, ReadTimeout

def grab_from_url_content(url):
    headers = {'Accept': '* / *',
               'Accept-Language': 'zh-TW, zh; q=0.9, en-US; q=0.8, en; q=0.7, zh-CN; q=0.6',
               'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3610.2 Safari/537.36'
               }
    rescontent = ''
    try:
        res = requests.get(url, headers=headers, timeout=10)
        rescontent = res.text
    except ConnectionError as ce:
        print('ConnectionError: ' + str(ce))
        return grab_from_url_content(url)
    except ReadTimeout as rte:
        print('ReadTimeout: ' + str(rte))
        return grab_from_url_content(url)

    return rescontent



## Parser

We will get the html content from the url which is not listed as we want it be, so we need parser to parse them into listed data, in json form.

In [9]:
from html.parser import HTMLParser

class DimensionsHTMLParser(HTMLParser):
    articles = []
    in_article = False
    is_next_trigger = False
    next_trigger_url = ''

    def handle_starttag(self, tag, attrs):
        if tag == 'article':
            self.in_article = True
        
        if self.in_article:
            for attr in attrs:
                if attr[0] == 'data-doc':
                    self.articles.append(json.loads(attr[1]))
                    break

        if tag == 'a':
            for attr in attrs:
                if (attr[0] == 'class') and (attr[1] == 'nextPage-trigger'):
                    self.is_next_trigger = True
                if self.is_next_trigger and (attr[0] == 'href'):
                    self.next_trigger_url = f'https://app.dimensions.ai{attr[1]}&{search_params}'
                    break
        return
                    
    def handle_endtag(self, tag):
        if tag == 'article':
            self.in_article = False
        self.is_next_trigger = False
        return

    def handle_data(self, data):
        pass

    def handle_comment(self, data):
        pass

    def handle_entityref(self, name):
        pass

    def handle_charref(self, name):
        pass

    def handle_decl(self, data):
        pass

parser = DimensionsHTMLParser()

## Grab Data

The data fetching starts from here.

In [10]:
%%time
for index, row in journals.iterrows():
    if index % 100 == 0:
        print(index)
    journal_search = row['search id']
    # This is very important for data fetching, generally you get url in the form as 'https://app.dimensions.ai/discover/publication?and_facet_for=2209&and_facet_for=2202&or_facet_source_title=jour.1082997', and search_params is just the substring after '?'.
    # Here is where we need to change for each search.
    search_params = f'or_facet_year=2014&or_facet_year=2015&or_facet_year=2016&or_facet_year=2017&or_facet_source_title={journal_search}'
    grab_url = f'https://app.dimensions.ai/discover/publication.contents.html?{search_params}'
    print(grab_url)

    parser.feed(grab_from_url_content(grab_url))
    next_trigger_anchor = ''
    i = 0
    while parser.next_trigger_url != next_trigger_anchor:
        if i % 100 == 0:
            print(str(i))
        i+=1
        next_trigger_anchor = parser.next_trigger_url
        parser.feed(grab_from_url_content(parser.next_trigger_url))

https://app.dimensions.ai/discover/publication.contents.html?or_facet_year=2014&or_facet_year=2015&or_facet_year=2016&or_facet_year=2017&or_facet_source_title=jour.1138469
0
https://app.dimensions.ai/discover/publication.contents.html?or_facet_year=2014&or_facet_year=2015&or_facet_year=2016&or_facet_year=2017&or_facet_source_title=jour.1137699
0
https://app.dimensions.ai/discover/publication.contents.html?or_facet_year=2014&or_facet_year=2015&or_facet_year=2016&or_facet_year=2017&or_facet_source_title=jour.1138183
0
https://app.dimensions.ai/discover/publication.contents.html?or_facet_year=2014&or_facet_year=2015&or_facet_year=2016&or_facet_year=2017&or_facet_source_title=jour.1136096
0
https://app.dimensions.ai/discover/publication.contents.html?or_facet_year=2014&or_facet_year=2015&or_facet_year=2016&or_facet_year=2017&or_facet_source_title=jour.1327791
0
https://app.dimensions.ai/discover/publication.contents.html?or_facet_year=2014&or_facet_year=2015&or_facet_year=2016&or_facet_yea

In [11]:
print(len(parser.articles))

2616


In [12]:
with open(f"{origin_folder}/articles_{journals_index}.json", "w") as dump_f:
        dump_f.write(json.dumps(parser.articles))

In [13]:
with open(f'{origin_folder}/articles_{journals_index}.json') as f:
    articles = json.load(f)

df_articles = pd.DataFrame(columns=['SO', 'SO_id', 'DI', 'dimensions_id'])

for article in articles:
    if 'doi' not in article:
        continue
    if article['doi'] in df_articles['DI']:
        continue
    df_article = pd.DataFrame(columns=['SO', 'SO_id', 'DI', 'dimensions_id'])
    df_article['SO'] = [article['source_title']]
    df_article['SO_id'] = [article['source_title_id']]
    df_article['DI'] = [article['doi']]
    df_article['dimensions_id'] = [article['id']]
    df_articles = df_articles.append(df_article, ignore_index=True)

df_articles.to_csv(f'{origin_folder}/articles_{journals_index}.csv', index=False)